##데이콘 태양광 발전략 예측 AI 경진대회 참여코드
-태양광 발전량 /시계열 /prinball Loss

-진행기간 20.12.09 ~21.01.06

-예시로 제시된 지역의 기상 데이터와 과거 발전량 데이터를 활용하여, 시간대별 태양광 발전량을 예측 


###대회 설명
-태양광 발전은 매일의 기상 상황과 계절에 따른 일사량의 영향을 받습니다. 이에 대한 예측이 가능하다면 보다 원활하게 전력 수급 계획을 세울 수 있습니다.

-신재생에너지의 생산 효율성을 극대화하고, 사용자들에게 저렴한 전력을 공급할 수 있도록 인공지능 기반 태양광 발전량 예측 모델을 만들어 주세요.  

-모델은 7일(Day 0~ Day6) 동안의 데이터를 인풋으로 활용하여, 향후 2일(Day7 ~ Day8) 동안의 30분 간격의 발전량(TARGET)을 예측해야 합니다. (1일당 48개씩 총 96개 타임스텝에 대한 예측)


주최: 연구개발특구진흥재단 / 한국원자력연구원(KAERI),  주관: 데이콘



 



In [ ]:
from google.colab import drive
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [ ]:
#모듈 import
import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#태양광 예측 데이터 로드
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_sunprediction/data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_sunprediction/data/sample_submission.csv')

###-칼럼설명
  Hour - 시간

  Minute - 분

  DHI - 수평면 산란일사량(Diffuse Horizontal Irradiance (W/m2))

  DNI - 직달일사량(Direct Normal Irradiance (W/m2))

  WS - 풍속(Wind Speed (m/s))

  RH - 상대습도(Relative Humidity (%))

  T - 기온(Temperature (Degree C))

  Target - 태양광 발전량 (kW)

In [ ]:

train.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0


In [ ]:
#데이터 통계 확인
train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Day,52560.0,547.000000,316.102148,0.00,273.0000,547.0,821.00000,1094.000000
Hour,52560.0,11.500000,6.922252,0.00,5.7500,11.5,17.25000,23.000000
Minute,52560.0,15.000000,15.000143,0.00,0.0000,15.0,30.00000,30.000000
DHI,52560.0,64.344121,103.897125,0.00,0.0000,0.0,87.00000,528.000000
DNI,52560.0,234.792371,349.684583,0.00,0.0000,0.0,469.00000,1059.000000
WS,52560.0,2.456033,1.426874,0.00,1.4000,2.2,3.20000,12.000000
RH,52560.0,56.793102,22.052875,7.59,39.6975,57.6,72.77000,100.000000
T,52560.0,9.279928,10.179741,-19.00,1.0000,9.0,17.00000,35.000000
TARGET,52560.0,17.790630,25.759955,0.00,0.0000,0.0,32.08989,99.913939


In [ ]:
print(train.shape)

(52560, 9)


###전처리 고려사항
데이터 분할

컬럼 변환

컬럼내에 특이값 제거 및 변경

데이터 정규화 : 평균 및 표준 편차는 모델이 검증 및 테스트 세트의 값에 액세스 할 수 없도록 훈련 데이터를 사용해서 만 계산되어야 한다.

데이터 윈도우 : 텐서플로우 튜토리얼 windowgenerator class 사용 input width =9Label width=2


In [ ]:
train.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0


In [ ]:
#데이터 컬럼에 다음날  그다음날  정답데이터 맞춰주기
def create_lag_feats(data, lags, cols):
    
    lag_cols = []
    temp = data.copy()
    for col in cols:
        for lag in lags:
            temp[col + '_lag_%s'%lag] = temp[col].shift(lag)
            temp['Target1'] = temp['TARGET']
            temp['Target2'] = temp['TARGET'].shift(-lag).fillna(method='ffill')  
            lag_cols.append(col + '_lag_%s'%lag)

    return temp, lag_cols
    

def preprocess_data(data, target_lags=[48], weather_lags=[48], is_train=True):
    
    temp = data.copy()

    if is_train==True:          
    
        temp, temp_lag_cols1 = create_lag_feats(temp, target_lags, ['TARGET'])
        temp, temp_lag_cols2 = create_lag_feats(temp, weather_lags, ['DHI', 'DNI', 'WS', 'RH', 'T'])
     
        return temp[['Hour'] + temp_lag_cols1 + temp_lag_cols2 + ['Target1', 'Target2']].dropna()

    elif is_train==False:    
        
        temp, temp_lag_cols1 = create_lag_feats(temp, target_lags, ['TARGET'])
        temp, temp_lag_cols2 = create_lag_feats(temp, weather_lags, ['DHI', 'DNI', 'WS', 'RH', 'T'])
                              
        return temp[['Hour'] + temp_lag_cols1 + temp_lag_cols2].dropna()


df_train = preprocess_data(train, target_lags=[48], weather_lags=[48], is_train=True)
df_train.iloc[:48]

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48,Target1,Target2
48,0,0.000000,0.0,0.0,1.5,69.08,-12.0,0.000000,0.000000
49,0,0.000000,0.0,0.0,1.5,69.06,-12.0,0.000000,0.000000
50,1,0.000000,0.0,0.0,1.6,71.78,-12.0,0.000000,0.000000
51,1,0.000000,0.0,0.0,1.6,71.75,-12.0,0.000000,0.000000
52,2,0.000000,0.0,0.0,1.6,75.20,-12.0,0.000000,0.000000
53,2,0.000000,0.0,0.0,1.5,69.29,-11.0,0.000000,0.000000
54,3,0.000000,0.0,0.0,1.5,72.56,-11.0,0.000000,0.000000
55,3,0.000000,0.0,0.0,1.4,72.55,-11.0,0.000000,0.000000
56,4,0.000000,0.0,0.0,1.3,74.62,-11.0,0.000000,0.000000
57,4,0.000000,0.0,0.0,1.3,74.61,-11.0,0.000000,0.000000


In [ ]:
df_train.head()

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48,Target1,Target2
48,0,0.0,0.0,0.0,1.5,69.08,-12.0,0.0,0.0
49,0,0.0,0.0,0.0,1.5,69.06,-12.0,0.0,0.0
50,1,0.0,0.0,0.0,1.6,71.78,-12.0,0.0,0.0
51,1,0.0,0.0,0.0,1.6,71.75,-12.0,0.0,0.0
52,2,0.0,0.0,0.0,1.6,75.20,-12.0,0.0,0.0


In [32]:
df_test =[]

for i in  range(81):
  file_path = '/content/drive/MyDrive/Colab Notebooks/dacon_sunprediction/data/test/'+str(i)+'.csv'
  temp = pd.read_csv(file_path)
  temp = preprocess_data(temp, target_lags=[48], weather_lags=[48], is_train=False).iloc[-48:]
  df_test.append(temp)

X_test = pd.concat(df_test)
X_test.shape
  

(23328, 7)

In [33]:
from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -2], test_size=0.3, random_state=0)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -1], test_size=0.3, random_state=0)

In [34]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [35]:
from lightgbm import LGBMRegressor

# Get the model and the predictions in (a) - (b)
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) Modeling  
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=10000, bagging_fraction=0.7, learning_rate=0.027, subsample=0.7)                   
                         
                         
    model.fit(X_train, Y_train, eval_metric = ['quantile'], 
          eval_set=[(X_valid, Y_valid)], early_stopping_rounds=300, verbose=500)

    # (b) Predictions
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [38]:
#타겟 예측
def train_data(X_train, Y_train, X_valid, Y_valid, X_test):
  LGBM_models =[]
  LGBM_actual_pred = pd.DataFrame()

  for q in quantiles :
    print(q)
    pred, model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
    LGBM_models.append(model)
    LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred], axis=1)
  
  LGBM_actual_pred.columns = quantiles

  return LGBM_models, LGBM_actual_pred

In [39]:
models_1, results_1 = train_data(X_train_1,Y_train_1,X_valid_1,Y_valid_1,X_test)


0.1
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's quantile: 1.37212
Early stopping, best iteration is:
[676]	valid_0's quantile: 1.37108
0.2
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's quantile: 2.16051
[1000]	valid_0's quantile: 2.14876
[1500]	valid_0's quantile: 2.14286
[2000]	valid_0's quantile: 2.13669
[2500]	valid_0's quantile: 2.13719
Early stopping, best iteration is:
[2316]	valid_0's quantile: 2.13589
0.3
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's quantile: 2.57354
[1000]	valid_0's quantile: 2.54514
[1500]	valid_0's quantile: 2.53945
[2000]	valid_0's quantile: 2.53633
[2500]	valid_0's quantile: 2.53497
[3000]	valid_0's quantile: 2.53294
[3500]	valid_0's quantile: 2.5275
Early stopping, best iteration is:
[3505]	valid_0's quantile: 2.5274
0.4
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's quantile: 2.6792
[1000]	valid_0's quantile: 2.65925
[1500

In [40]:
models_2, results_2 = train_data(X_train_2,Y_train_2,X_valid_2,Y_valid_2,X_test)

0.1
Training until validation scores don't improve for 300 rounds.
Early stopping, best iteration is:
[182]	valid_0's quantile: 1.4093
0.2
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's quantile: 2.2503
[1000]	valid_0's quantile: 2.23588
Early stopping, best iteration is:
[1173]	valid_0's quantile: 2.23198
0.3
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's quantile: 2.66486
[1000]	valid_0's quantile: 2.64927
[1500]	valid_0's quantile: 2.63853
Early stopping, best iteration is:
[1398]	valid_0's quantile: 2.63812
0.4
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's quantile: 2.8087
[1000]	valid_0's quantile: 2.77833
[1500]	valid_0's quantile: 2.76475
Early stopping, best iteration is:
[1651]	valid_0's quantile: 2.76252
0.5
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's quantile: 2.72555
[1000]	valid_0's quantile: 2.6931
Early stopping, best iteration is:
[1174]	va

In [43]:
results_1
resulst

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
23323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values

ValueError: ignored

In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/dacon_sunprediction/data/submission2.csv', index=False)